In [10]:
# %% [markdown]
# # Smart City Traffic & Accident Analytics ETL Process with dim_time
#
# This notebook:
# 1. Loads sensor and accident CSV data.
# 2. Performs data quality checks (uniqueness of primary keys and missing data).
# 3. Cleans and transforms the data (e.g., converting date columns, standardizing column names).
# 4. Creates dimension tables:
#    - dim_time
#    - dim_location
#    - dim_vehicle
#    - dim_weather
#    - dim_road
# 5. Creates fact tables:
#    - fact_traffic
#    - fact_accident
# 6. Loads the resulting tables into a MySQL database using SQLAlchemy.
#
# **Note**: The MySQL connection details are:
#
# username = 'root'
# password = '12345'
# host = 'localhost'
# port = '3306'
# database = 'case1'
#
# Table names are created in lower-case.

# %% [code]
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# %% [markdown]
# ## 1. Load CSV Files
#
# Read the CSV files into pandas DataFrames and standardize column names.

# %% [code]
# Load CSV files
df_sensor = pd.read_csv('road_traffic_sensor_data.csv')
df_accident = pd.read_csv('traffic_accident_data.csv')

# Standardize column names to lower-case
df_sensor.columns = [col.lower() for col in df_sensor.columns]
df_accident.columns = [col.lower() for col in df_accident.columns]

# %% [markdown]
# ## 2. Data Quality Checks & Cleaning
#
# ### 2.1 Check for Uniqueness of Primary Keys

# %% [code]
# For sensor data, the primary key is sensor_id
if df_sensor['sensor_id'].nunique() != len(df_sensor):
    print("Warning: Duplicate sensor_id values found in sensor data!")
else:
    print("All sensor_id values are unique in sensor data.")

# For accident data, the primary key is accident_id
if df_accident['accident_id'].nunique() != len(df_accident):
    print("Warning: Duplicate accident_id values found in accident data!")
else:
    print("All accident_id values are unique in accident data.")

# %% [markdown]
# ### 2.2 Check for Missing Data

# %% [code]
print("Missing values in sensor data:")
print(df_sensor.isnull().sum())

print("\nMissing values in accident data:")
print(df_accident.isnull().sum())

# Drop rows with missing values (or alternatively, you can impute them)
df_sensor.dropna(inplace=True)
df_accident.dropna(inplace=True)

# %% [markdown]
# ### 2.3 Convert Data Types
#
# Convert the **date_time** columns to datetime objects.

# %% [code]
df_sensor['date_time'] = pd.to_datetime(df_sensor['date_time'])
df_accident['date_time'] = pd.to_datetime(df_accident['date_time'])

# %% [markdown]
# ## 3. Build Dimension Tables
#
# ### 3.1 dim_time
#
# We combine unique date_time values from both sensor and accident datasets.  
# Then we extract additional time attributes (year, month, day, hour, etc.).

# %% [code]
# Combine unique date_time values from sensor and accident data
all_times = pd.concat([df_sensor[['date_time']], df_accident[['date_time']]])
all_times = all_times.drop_duplicates().reset_index(drop=True)

# Extract time attributes
all_times['year'] = all_times['date_time'].dt.year
all_times['month'] = all_times['date_time'].dt.month
all_times['day'] = all_times['date_time'].dt.day
all_times['hour'] = all_times['date_time'].dt.hour
all_times['day_of_week'] = all_times['date_time'].dt.dayofweek  # Monday=0, Sunday=6

# Create a surrogate key for time_id (starting at 1)
all_times.reset_index(inplace=True)
all_times.rename(columns={'index': 'time_id'}, inplace=True)
all_times['time_id'] = all_times['time_id'] + 1

dim_time = all_times[['time_id', 'date_time', 'year', 'month', 'day', 'hour', 'day_of_week']]
print("dim_time:")
print(dim_time.head())

# %% [markdown]
# ### 3.2 dim_location
#
# Consolidate distinct locations from both datasets.

# %% [code]
locations_sensor = df_sensor[['location']].drop_duplicates()
locations_accident = df_accident[['location']].drop_duplicates()
all_locations = pd.concat([locations_sensor, locations_accident]).drop_duplicates().reset_index(drop=True)

# Surrogate key for location_id (starting at 1)
all_locations.reset_index(inplace=True)
all_locations.rename(columns={'index': 'location_id'}, inplace=True)
all_locations['location_id'] = all_locations['location_id'] + 1

dim_location = all_locations[['location_id', 'location']]
print("\ndim_location:")
print(dim_location.head())

# %% [markdown]
# ### 3.3 Additional Dimensions for Accident Data
#
# Create dimensions for vehicle type, weather condition, and road condition.

# %% [code]
# Vehicle Dimension
dim_vehicle = df_accident[['vehicle_type']].drop_duplicates().reset_index(drop=True)
dim_vehicle.reset_index(inplace=True)
dim_vehicle.rename(columns={'index': 'vehicle_id'}, inplace=True)
dim_vehicle['vehicle_id'] = dim_vehicle['vehicle_id'] + 1

dim_vehicle = dim_vehicle[['vehicle_id', 'vehicle_type']]
print("\ndim_vehicle:")
print(dim_vehicle.head())

# Weather Dimension
dim_weather = df_accident[['weather_condition']].drop_duplicates().reset_index(drop=True)
dim_weather.reset_index(inplace=True)
dim_weather.rename(columns={'index': 'weather_id'}, inplace=True)
dim_weather['weather_id'] = dim_weather['weather_id'] + 1

dim_weather = dim_weather[['weather_id', 'weather_condition']]
print("\ndim_weather:")
print(dim_weather.head())

# Road Condition Dimension
dim_road = df_accident[['road_condition']].drop_duplicates().reset_index(drop=True)
dim_road.reset_index(inplace=True)
dim_road.rename(columns={'index': 'road_id'}, inplace=True)
dim_road['road_id'] = dim_road['road_id'] + 1

dim_road = dim_road[['road_id', 'road_condition']]
print("\ndim_road:")
print(dim_road.head())

# %% [markdown]
# ## 4. Build Fact Tables
#
# ### 4.1 fact_traffic
#
# - Merge sensor data with `dim_time` on `date_time`.
# - Merge sensor data with `dim_location` on `location`.
# - Keep only the surrogate keys (`fk_time_id`, `fk_location_id`), the primary key (`sensor_id`), and the relevant numeric or measure columns.

# %% [code]
# Merge with dim_time to get fk_time_id
fact_traffic = df_sensor.merge(dim_time[['time_id', 'date_time']], on='date_time', how='left')
fact_traffic.rename(columns={'time_id': 'fk_time_id'}, inplace=True)

# Merge with dim_location to get fk_location_id
fact_traffic = fact_traffic.merge(dim_location, on='location', how='left')
fact_traffic.rename(columns={'location_id': 'fk_location_id'}, inplace=True)

# Select only relevant columns
fact_traffic = fact_traffic[['sensor_id', 'fk_time_id', 'fk_location_id',
                             'vehicle_count', 'average_speed', 'congestion_level']]

print("\nfact_traffic:")
print(fact_traffic.head())

# %% [markdown]
# ### 4.2 fact_accident
#
# - Merge accident data with `dim_time`, `dim_location`, `dim_vehicle`, `dim_weather`, and `dim_road`.
# - Keep only the surrogate keys (e.g., `fk_time_id`, `fk_location_id`, etc.), the primary key (`accident_id`), and measure columns.

# %% [code]
# Merge with dim_time
fact_accident = df_accident.merge(dim_time[['time_id', 'date_time']], on='date_time', how='left')
fact_accident.rename(columns={'time_id': 'fk_time_id'}, inplace=True)

# Merge with dim_location
fact_accident = fact_accident.merge(dim_location, on='location', how='left')
fact_accident.rename(columns={'location_id': 'fk_location_id'}, inplace=True)

# Merge with dim_vehicle
dim_vehicle_renamed = dim_vehicle.rename(columns={'vehicle_id': 'fk_vehicle_id'})
fact_accident = fact_accident.merge(dim_vehicle_renamed, on='vehicle_type', how='left')

# Merge with dim_weather
dim_weather_renamed = dim_weather.rename(columns={'weather_id': 'fk_weather_id'})
fact_accident = fact_accident.merge(dim_weather_renamed, on='weather_condition', how='left')

# Merge with dim_road
dim_road_renamed = dim_road.rename(columns={'road_id': 'fk_road_id'})
fact_accident = fact_accident.merge(dim_road_renamed, on='road_condition', how='left')

# Select only necessary columns (no redundancy)
fact_accident = fact_accident[['accident_id', 'fk_time_id', 'fk_location_id',
                               'fk_vehicle_id', 'fk_weather_id', 'fk_road_id',
                               'accident_severity', 'number_of_vehicles', 'casualties',
                               'traffic_density']]

print("\nfact_accident:")
print(fact_accident.head())

# %% [markdown]
# ## 5. Load Tables into MySQL
#
# Using SQLAlchemy, we connect to the MySQL database and load each dimension and fact table.
# All table names are in lower-case.

# %% [code]
# MySQL connection parameters
username = 'root'
password = '12345'
host = 'localhost'
port = '3306'
database = 'case6'

# Create the SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# Load dimension and fact tables to MySQL (if_exists='replace' will overwrite existing tables)
dim_time.to_sql('dim_time', con=engine, index=False, if_exists='replace')
dim_location.to_sql('dim_location', con=engine, index=False, if_exists='replace')
dim_vehicle.to_sql('dim_vehicle', con=engine, index=False, if_exists='replace')
dim_weather.to_sql('dim_weather', con=engine, index=False, if_exists='replace')
dim_road.to_sql('dim_road', con=engine, index=False, if_exists='replace')
fact_traffic.to_sql('fact_traffic', con=engine, index=False, if_exists='replace')
fact_accident.to_sql('fact_accident', con=engine, index=False, if_exists='replace')

print("\nAll tables have been loaded successfully into MySQL!")


All sensor_id values are unique in sensor data.
All accident_id values are unique in accident data.
Missing values in sensor data:
sensor_id           0
location            0
date_time           0
vehicle_count       0
average_speed       0
congestion_level    0
dtype: int64

Missing values in accident data:
accident_id           0
date_time             0
location              0
weather_condition     0
road_condition        0
vehicle_type          0
accident_severity     0
number_of_vehicles    0
casualties            0
traffic_density       0
dtype: int64
dim_time:
   time_id           date_time  year  month  day  hour  day_of_week
0        1 2024-01-01 00:00:00  2024      1    1     0            0
1        2 2024-01-01 01:00:00  2024      1    1     1            0
2        3 2024-01-01 02:00:00  2024      1    1     2            0
3        4 2024-01-01 03:00:00  2024      1    1     3            0
4        5 2024-01-01 04:00:00  2024      1    1     4            0

dim_location:
   l